In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from datetime import datetime

Proposed Datasets
Show Info:
Season
Episode
Air date
Contestant 1
Contestant 2
Contestant 3
Returning Contestant

Show Details:
Episode
Round
Category
Clue
Value
Clue Order
Correct Response
Correct Responder
Incorrect Response 1
Incorrect Responder 1
Incorrect Response 2
Incorrect Responder 2
Incorrect Response 3
Incorrect Responder 3
Daily Double
Wager

Show Statistics:
J Round Score C1
J Round Score C2
J Round Score C3
DJ Round Score C1
DJ Round Score C2
DJ Round Score C3
FJ Wager C1
FJ Wager C2
FJ Wager C3
FJ Round Score C1
FJ Round Score C2
FJ Round Score C3


All Jeopardy! seasons:
http://www.j-archive.com/listseasons.php

## Season List

In [2]:
response = requests.get('http://www.j-archive.com/listseasons.php')

In [3]:
soup = BeautifulSoup(response.text)
#soup

In [5]:
season_urls = np.array(['http://www.j-archive.com/' + a['href'] for a in soup.findAll('a') 
                    if 'showseason' in a['href'] and 'http' not in a['href']])
#season_urls

In [6]:
seasons = pd.DataFrame(season_urls)
seasons.columns = ['season_url']
#seasons
seasons.to_csv('data/j_seasons.csv', index=False)

## Episode List

In [7]:
response = requests.get('http://www.j-archive.com/showseason.php?season=37')

In [8]:
soup = BeautifulSoup(response.text)
#soup

In [9]:
episodes = np.array([a['href'] for a in soup.findAll('a') if 'game_id' in a['href']])
episodes = pd.DataFrame(episodes)
episodes.columns = ['episode_url']

In [10]:
episodes.to_csv('data/j_episodes.csv', index=False)

## Episode Details

In [34]:
#response = requests.get('http://www.j-archive.com/showgame.php?game_id=6895')
#response = requests.get('http://www.j-archive.com/showgame.php?game_id=6893')
response = requests.get('http://www.j-archive.com/showgame.php?game_id=6880')

In [35]:
soup = BeautifulSoup(response.text)
#soup

In [36]:
header = soup.find('h1')
header = header.text
show_num = header.split('-')[0].strip().split('#')[1]
air_date = datetime.strftime(datetime.strptime(header.split('-')[1].strip().split(',',1)[1].strip(),'%B %d, %Y'), '%Y-%m-%d')
#print(show_num, ', ', air_date)

In [37]:
season = soup.find('a', text='[current season]')
print(season)
season = season['href'].split('=')[1]

<a href="http://www.j-archive.com/showseason.php?season=37">[current season]</a>


In [38]:
scores = soup.findAll(['h3','table'])
score_texts = np.array([['Scores at the end of the Jeopardy! Round:', 'J'],
                        ['Scores at the end of the Double Jeopardy! Round:', 'DJ'],
                        ['Final scores:', 'FJ']])
j_round = []
j_name = []
j_score = []
for x in score_texts:
    for element in scores:
        if element.text == x[0]:
            while element.name != 'table':
                element = element.next_sibling
                #print('name: ',element.name)
            score_table = element.findAll('tr')
            [j_round.append(x[1]) for td in score_table[0].findAll('td')]
            [j_name.append(td.text) for td in score_table[0].findAll('td')]
            [j_score.append(td.text) for td in score_table[1].findAll('td')]

j_scores = pd.DataFrame({'show_num': show_num, 'round_code': j_round, 'contestant': j_name, 'score': j_score})
j_scores

,show_num,round_code,contestant,score
0,8290,J,Ryan,"$4,800"
1,8290,J,Gabriel,"$3,200"
2,8290,J,Chandy,-$200
3,8290,DJ,Ryan,"$19,400"
4,8290,DJ,Gabriel,"$14,700"
5,8290,DJ,Chandy,"$12,600"
6,8290,FJ,Ryan,"$29,401"
7,8290,FJ,Gabriel,$2
8,8290,FJ,Chandy,"$8,200"


In [39]:
remarks_text = 'Coryat scores:'
j_right = []
j_ddright = []
j_wrong = []
j_ddwrong = []
for element in scores:
    if element.text == remarks_text:
        while element.name != 'table':
            element = element.next_sibling
        remarks_table = element.findAll('td', class_ = 'score_remarks')
        [j_right.append((re.findall('(\d{1,2}) R', x.text.split(',')[0]) or list('0'))[0]) for x in remarks_table]
        [j_ddright.append((re.findall('(\d{1}) DD', x.text.split(',')[0]) or list('0'))[0]) for x in remarks_table]
        [j_wrong.append((re.findall('(\d{1,2}) W', x.text.split(',')[1]) or list('0'))[0]) for x in remarks_table]
        [j_ddwrong.append((re.findall('(\d{1}) DD', x.text.split(',')[1]) or list('0'))[0]) for x in remarks_table]

j_remarks = pd.DataFrame({'contestant': j_name[:3], 'num_right': j_right, 'num_ddright': j_ddright, 'num_wrong': j_wrong,
                         'num_ddwrong': j_ddwrong})
#j_remarks

In [40]:
j_scores = j_scores.merge(j_remarks, how = 'left', on='contestant')
#j_scores

In [55]:
finish_order = []
winner = j_scores[j_scores.round_code=='FJ'].sort_values(['score'], ascending = False)['contestant'].iloc[0]
second = j_scores[j_scores.round_code=='FJ'].sort_values(['score'], ascending = False)['contestant'].iloc[1]
third = j_scores[j_scores.round_code=='FJ'].sort_values(['score'], ascending = False)['contestant'].iloc[2]
finish_order.append([winner, 'Winner'])
finish_order.append([second, 'Second'])
finish_order.append([third, 'Third'])
finish_order = pd.DataFrame(finish_order)
finish_order.columns=(['contestant', 'finish'])
j_scores = j_scores.merge(finish_order, how = 'left', on = 'contestant')
j_scores.to_csv('data/j_scores.csv', index=False, header=False)

In [54]:
#j_scores

,show_num,round_code,contestant,score,num_right,num_ddright,num_wrong,num_ddwrong,finish
0,8290,J,Ryan,"$4,800",16,1,1,0,Second
1,8290,J,Gabriel,"$3,200",26,1,6,1,Third
2,8290,J,Chandy,-$200,10,0,1,0,Winner
3,8290,DJ,Ryan,"$19,400",16,1,1,0,Second
4,8290,DJ,Gabriel,"$14,700",26,1,6,1,Third
5,8290,DJ,Chandy,"$12,600",10,0,1,0,Winner
6,8290,FJ,Ryan,"$29,401",16,1,1,0,Second
7,8290,FJ,Gabriel,$2,26,1,6,1,Third
8,8290,FJ,Chandy,"$8,200",10,0,1,0,Winner


In [42]:
categories = soup.findAll('td', attrs = {'class': 'category_name'})

In [43]:
categories = np.array([x.text for x in categories])
#categories

In [44]:
rounds = []
[[rounds.append(item) for x in range(6)] for item in ['J', 'DJ']]
rounds.append('FJ')
rounds = np.array(rounds)
#rounds

In [45]:
catnums = np.array(['1', '2', '3', '4', '5', '6', '1', '2', '3', '4', '5', '6', '1'])
#catnums

In [46]:
categories = pd.DataFrame({'category': categories, 'round_code': rounds, 'cat_num': catnums})

In [47]:
categories['season'] = season
categories['show_num'] = show_num
categories['air_date'] = air_date
#categories

In [48]:
#split the results by the "clue" class
clues = soup.find_all('td', attrs = {'class': 'clue'})
#print(type(clues))
#print(type(soup))

In [49]:
all_rows = []
for clue in clues:
    #print(type(clue))
    new_row = []
    #Get our clue number, which tells us the round, category, and board position
    clue_id = clue.find('td', class_='clue_text')
    if not clue_id == None:
        #print(clue_id)
        clue_id = clue.find('td', class_='clue_text')['id']
        #What round is this?
        round_code = clue_id.split('_')[1]
        cat_num = clue_id.split('_')[2] if not round_code == 'FJ' else 'checkme'
        clue_num = clue_id.split('_')[3] if not round_code == 'FJ' else 'checkme'
        if not round_code == 'FJ':
            #Get our clue value
            #DD does not have a clue_value, it has a clue_value_daily_double
            #For DD split out the dollar value and set the flag
            clue_value = clue.find('td', class_='clue_value_daily_double').text.split(' ')[1] if not clue.find('td', {'class':'clue_value'}) else clue.find('td', {'class':'clue_value'}).text
            daily_double = False if not clue.find('td', class_='clue_value_daily_double') else True
            clue_order = clue.find('td', class_='clue_order_number').text
            clue_text = clue.find('td', class_='clue_text').text
            corr_resp = BeautifulSoup(clue.find('div', onmouseover=True).get('onmouseover'))
            #print(corr_resp)
            #Check for clues with no correct responder, and set the flag
            corr_respr1 = '' if not corr_resp.find('td', class_='right') else corr_resp.find('td', class_='right').text
            corr_FJ_wager1 = ''
            corr_respr2 = ''
            corr_FJ_wager2 = ''
            corr_respr3 = ''
            corr_FJ_wager3 = ''
            triple_stumper = True if not corr_resp.find('td', class_='right') else False
            incorr_resprs = corr_resp.findAll('td', class_='wrong')
            #Okay, THIS is ugly
            incorr_respr1 = ''
            incorr_FJ_wager1 = ''
            incorr_respr2 = ''
            incorr_FJ_wager2 = ''
            incorr_respr3 = ''
            incorr_FJ_wager3 = ''
            if len(incorr_resprs) >= 1:
                incorr_respr1 = '' if incorr_resprs[0].text == 'Triple Stumper' else incorr_resprs[0].text
            if len(incorr_resprs) >= 2:
                incorr_respr2 = '' if incorr_resprs[1].text == 'Triple Stumper' else incorr_resprs[1].text
            if len(incorr_resprs) == 3:
                incorr_respr3 = '' if incorr_resprs[2].text == 'Triple Stumper' else incorr_resprs[2].text
            corr_resp = corr_resp.find('em', class_='correct_response').text
            new_row = [round_code, cat_num, clue_num, clue_id, clue_value, daily_double, clue_order, clue_text, corr_resp, 
                       corr_respr1, corr_FJ_wager1, corr_respr2, corr_FJ_wager2, corr_respr3, corr_FJ_wager3, 
                       triple_stumper, incorr_respr1, incorr_FJ_wager1, incorr_respr2, incorr_FJ_wager2, incorr_respr3, incorr_FJ_wager3 ]
            all_rows.append(new_row)


<td class="clue_text" id="clue_J_1_1">In 1979 this horror master said, "Most of the market for my stuff is in paperbacks", but he's done okay in hardcover, too</td>
<td class="clue_text" id="clue_J_2_1">The name of this system that's used to detect &amp; locate objects is a palindrome &amp; an acronym</td>
<td class="clue_text" id="clue_J_3_1">Stuff the Magic Dragon:<br/>This NBA city</td>
<td class="clue_text" id="clue_J_4_1">It's a portable radio transmitter &amp; receiver for 2-way communication</td>
<td class="clue_text" id="clue_J_5_1">99.9% of all U.S. businesses, small businesses are defined as having fewer than this many employees, D to the Romans</td>
<td class="clue_text" id="clue_J_6_1">You get really high up hang-gliding over Bogota, this country's capital--already pretty high up at nearly 9,000 feet</td>
<td class="clue_text" id="clue_J_1_2">Crush your enemies &amp; name this Robert E. Howard barbarian whose "Collected Adventures" are collected in paperback</td>
<td class=

In [50]:
all_rows

[['J',
  '1',
  '1',
  'clue_J_1_1',
  '$200',
  False,
  '30',
  'In 1979 this horror master said, "Most of the market for my stuff is in paperbacks", but he\'s done okay in hardcover, too',
  '(Stephen) King',
  'Gabriel',
  '',
  '',
  '',
  '',
  '',
  False,
  '',
  '',
  '',
  '',
  '',
  ''],
 ['J',
  '2',
  '1',
  'clue_J_2_1',
  '$200',
  False,
  '27',
  "The name of this system that's used to detect & locate objects is a palindrome & an acronym",
  'radar',
  'Gabriel',
  '',
  '',
  '',
  '',
  '',
  False,
  '',
  '',
  '',
  '',
  '',
  ''],
 ['J',
  '3',
  '1',
  'clue_J_3_1',
  '$200',
  False,
  '5',
  'Stuff the Magic Dragon:This NBA city',
  'Orlando',
  'Gabriel',
  '',
  '',
  '',
  '',
  '',
  False,
  '',
  '',
  '',
  '',
  '',
  ''],
 ['J',
  '4',
  '1',
  'clue_J_4_1',
  '$200',
  False,
  '28',
  "It's a portable radio transmitter & receiver for 2-way communication",
  'a walkie-talkie',
  'Ryan',
  '',
  '',
  '',
  '',
  '',
  False,
  '',
  '',
  '',
  '',

In [51]:
round_code = 'FJ'
cat_num = '1'
clue_num = '1'
clue_id = 'clue_FJ'
clue_value = ''
daily_double = False
clue_order = ''
clue_text = ''
corr_resp = ''
triple_stumper = True

final_round = soup.find('table', class_ = 'final_round').find_all('td')
#type(final_round)
#final_round - this will absolutely barf and so will a print; it gives a type error

for item in final_round:
    #this only works because it's the first item! aaargh
    stats = BeautifulSoup(item.find('div', onmouseover=True).get('onmouseover'))
    if stats:
        break
corr_resp = stats.find('em')
corr_resp = corr_resp.get_text()

#make a dictionary of responders
resprs = {}
wagers = stats.findAll('td')

right = 0
wrong = 0
for td in wagers:
    td_text = td.get_text()
    if td.has_attr('class'):
        td_class = td.get('class')[0]
        if td_class == 'right':
            triple_stumper = False
            right +=1
            key_name = 'corr_respr' + str(right)
            resprs[key_name] = td_text
            wager_key = 'corr_FJ_wager' + str(right)
        else:
            wrong +=1
            key_name = 'incorr_respr' + str(wrong)
            resprs[key_name] = td_text
            wager_key = 'incorr_FJ_wager' + str(wrong)
    if td.has_attr('rowspan'):
        continue
    if not td.has_attr('class'):
        resprs[wager_key] = td_text
        
corr_respr1 = resprs.get('corr_respr1', '')
corr_FJ_wager1 = resprs.get('corr_FJ_wager1', '')
corr_respr2 = resprs.get('corr_respr2', '')
corr_FJ_wager2 = resprs.get('corr_FJ_wager2', '')
corr_respr3 = resprs.get('corr_respr3', '')
corr_FJ_wager3 = resprs.get('corr_FJ_wager3', '')
incorr_respr1 = resprs.get('incorr_respr1','')
incorr_FJ_wager1 = resprs.get('incorr_FJ_wager1', '')
incorr_respr2 = resprs.get('incorr_respr2','')
incorr_FJ_wager2 = resprs.get('incorr_FJ_wager2', '')
incorr_respr3 = resprs.get('incorr_respr3','')
incorr_FJ_wager3 = resprs.get('incorr_FJ_wager3', '')

clue_text = soup.find('td', attrs = {'class': 'clue_text', 'id': 'clue_FJ'})
clue_text = clue_text.get_text()
#print(clue_text)

new_row = [round_code, cat_num, clue_num, clue_id, clue_value, daily_double, clue_order, clue_text, corr_resp, 
                   corr_respr1, corr_FJ_wager1, corr_respr2, corr_FJ_wager2, corr_respr3, corr_FJ_wager3, 
                   triple_stumper, incorr_respr1, incorr_FJ_wager1, incorr_respr2, incorr_FJ_wager2, incorr_respr3, incorr_FJ_wager3]
all_rows.append(new_row)

In [52]:
this_show = pd.DataFrame(all_rows)
this_show.columns = ['round_code', 'cat_num', 'clue_num', 'clue_id', 'clue_value', 'daily_double', 'clue_order', 'clue_text', 'corr_resp', 
                   'corr_respr1', 'corr_FJ_wager1', 'corr_respr2', 'corr_FJ_wager2', 'corr_respr3', 'corr_FJ_wager3', 
                   'triple_stumper', 'incorr_respr1', 'incorr_FJ_wager1', 'incorr_respr2', 'incorr_FJ_wager2', 'incorr_respr3', 'incorr_FJ_wager3']
this_show = this_show.merge(categories, how='left', left_on=['round_code', 'cat_num'], right_on=['round_code', 'cat_num'])
this_show

,round_code,cat_num,clue_num,clue_id,clue_value,daily_double,clue_order,clue_text,corr_resp,corr_respr1,...,incorr_respr1,incorr_FJ_wager1,incorr_respr2,incorr_FJ_wager2,incorr_respr3,incorr_FJ_wager3,category,season,show_num,air_date
0,J,1,1,clue_J_1_1,$200,False,30,"In 1979 this horror master said, ""Most of the ...",(Stephen) King,Gabriel,...,,,,,,,PAPERBACK WRITER,37,8290,2020-11-27
1,J,2,1,clue_J_2_1,$200,False,27,The name of this system that's used to detect ...,radar,Gabriel,...,,,,,,,PALINDROMES,37,8290,2020-11-27
2,J,3,1,clue_J_3_1,$200,False,5,Stuff the Magic Dragon:This NBA city,Orlando,Gabriel,...,,,,,,,PRO TEAMS' MASCOTS,37,8290,2020-11-27
3,J,4,1,clue_J_4_1,$200,False,28,It's a portable radio transmitter & receiver f...,a walkie-talkie,Ryan,...,,,,,,,REAL-DEAL RHYME TIME,37,8290,2020-11-27
4,J,5,1,clue_J_5_1,$200,False,29,"99.9% of all U.S. businesses, small businesses...",500,Gabriel,...,,,,,,,LET'S START A SMALL BUSINESS,37,8290,2020-11-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,DJ,3,5,clue_DJ_3_5,$2000,False,27,"On a commercial flight, you'll spend most of y...",the troposphere,Gabriel,...,,,,,,,THE ATMOSPHERE,37,8290,2020-11-27
57,DJ,4,5,clue_DJ_4_5,$2000,False,18,"Deposed in 1797, Ludovico Manin was the last o...",the doges,Ryan,...,,,,,,,RANKS & TITLES,37,8290,2020-11-27
58,DJ,5,5,clue_DJ_5_5,$2000,False,19,Miriam was the sister of these 2 important fig...,Aaron & Moses,Chandy,...,,,,,,,BIBLICAL WOMEN,37,8290,2020-11-27
59,DJ,6,5,clue_DJ_6_5,$2000,False,12,It's a question & answer method of religious i...,catechism,Chandy,...,,,,,,,"C AS IN ""CAT""",37,8290,2020-11-27


In [53]:
this_show.to_csv('data/j_clues.csv', index=False)

In [ ]:
#this_show.to_csv('data/j_clues.csv', mode='a', index=False, header=False)

https://github.com/whymarrh/jeopardy-parser/blob/master/parser.py